In [22]:
# from init import *
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import gif

time: 1 ms


In [20]:
@gif.frame
def plot(x, y, T, i):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_title(i)
    ax.axis('off')
    levels = np.linspace(-10, 10, 30)
    im = ax.contourf(x, y, T, cmap='gray', levels=levels)
    plt.colorbar(im, ticks=np.arange(-10, 11, 5))

def pol2cart(q, r):
    x = r * np.cos(q)
    y = r * np.sin(q)
    return x, y

time: 2 ms


In [3]:
r1 = 0
q1 = 0
alpha = 10
Qv = 333000

Ta = 10
Tc = -10
Tc_l = 0.01

rho = 925
rho_l = 998.2

k = 2.219
k_l = 0.6

Cp = 2.1 * 1000
Cp_l = 4.18 * 1000

R = (0.75 * 0.0001 * 0.5) / np.cos(np.pi / 10.0)
delta_t = 0.01
delta_r = R / 48
delta_q = np.pi / 20

S = alpha * delta_r / k
S_l = alpha * delta_r / k_l
U = (k * delta_t) / (rho * Cp)
U_l = (k_l * delta_t) / (rho_l * Cp_l)
M = rho_l * (2 * np.pi * R**3) / 3.0

time: 158 ms


In [4]:
Ts = 30000
T = np.zeros((49, 21, Ts))
T[:49, [0, 20], :Ts] = Tc
T[1:49, 1:20, 0] = Ta

for n in tqdm(np.arange(Ts-2)):
    for i in np.arange(1, 48):
        for j in np.arange(1, 20):
            ml = U_l * 1.0 / (r1 + (i-1) * delta_r * (delta_q**2))
            up = T[i, j+1, n] + T[i, j-1, n] - 2 * T[i, j, n]
            T[i, j, n+1] = T[i, j, n] + ml * up

            if T[i, j, n+1] == 0:
                up1 = T[i, j+1, n+1] + T[i, j-1, n+1] - 2 * T[i, j, n+1]
                T[i, j, n+2] = T[i, j, n+1] + (ml / U_l) * U * up1 + (Qv*delta_t) / (rho*Cp)

            elif T[i, j, n+1] < 0:
                up1 = T[i, j+1, n+1] + T[i, j-1, n+1] - 2 * T[i, j, n+1]
                T[i, j, n+2] = T[i, j, n+1] + (ml / U_l) * U * up1

        if (T[i, 10, n] > -0.001) & (T[i, 10, n] < .001):
            m = i * delta_r
            a = np.sqrt(np.abs(R ** 2 - m ** 2))
            # freezing front
            M_1 = rho * np.pi * ((R ** 2) * m - (m ** 3) / 3.0)
            # Mass above freezing front
            M_2 = M - M_1                                                                           
            P = np.sqrt(((6 * M_2) / (2 * np.pi * rho_l)) ** 2 + a ** 3)
            A = 6 * M_2 / (2 * np.pi * rho_l) + P
            B = 6 * M_2 / (2 * np.pi * rho_l) - P
            h = A ** (1.0 / 3.0) + B ** (1.0 / 3.0)

100%|██████████| 29998/29998 [07:34<00:00, 65.95it/s]

time: 7min 34s


In [21]:
q = np.linspace(0, np.pi, 21)
r = np.linspace(0, R, 49)

[q, r] = np.meshgrid(q, r)
[x, y] = pol2cart(q, r)

frames = []

for fr in tqdm(np.arange(0, Ts, 2000)):
    frame = plot(x, y, T[:, :, fr], fr)
    frames.append(frame)

gif.save(frames, 'droplet.gif', duration=300)

100%|██████████| 15/15 [00:02<00:00,  7.01it/s]


time: 2.69 s
